# TDE Classification Pipeline v11

Cleaned version with TOP 100 features and parallel processing.


## SECTION 1: IMPORTS AND CONFIGURATION

In [1]:
import os
from pathlib import Path
import random
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from scipy.optimize import curve_fit
from scipy.stats import rankdata, pearsonr
from scipy.ndimage import gaussian_filter1d
import math

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    f1_score, precision_score, recall_score,
    roc_auc_score, average_precision_score, confusion_matrix
)

import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier

from joblib import Parallel, delayed

try:
    from numba import jit
    HAS_NUMBA = True
except ImportError:
    HAS_NUMBA = False
    def jit(*args, **kwargs):
        def decorator(func):
            return func
        return decorator

# Configuration
SEED = 42
N_FOLDS = 5
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SIGMA_THRESHOLD = 1.0  # Detection threshold

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(SEED)


FEATURES_TO_REMOVE = []


# Model caching configuration
CACHE_DIR = Path("/kaggle/input/mallorenlargemodels")#Path("/kaggle/working/")
CACHE_DIR.mkdir(exist_ok=True)
USE_CACHED_SEQ_MODELS = True  # Set False to force retraining
SAVE_SEQ_MODELS = True

# SNCOSMO features path
SNCOSMO_TRAIN_PATH = Path("/kaggle/input/sncosmos/train_sncosmo_features.parquet")
SNCOSMO_TEST_PATH = Path("/kaggle/input/sncosmos/test_sncosmo_features.parquet")
# Fallback to working directory
if not SNCOSMO_TRAIN_PATH.exists():
    SNCOSMO_TRAIN_PATH = Path("/kaggle/working/train_sncosmo_features.parquet")
    SNCOSMO_TEST_PATH = Path("/kaggle/working/test_sncosmo_features.parquet")

# SN Ia Veto
USE_SNIA_VETO = True  # Apply hard veto for objects with good SN Ia fits

print(f"Use SN Ia veto: {USE_SNIA_VETO}")

Use SN Ia veto: True


## SECTION 2: DATA LOADING

In [2]:
DATASET_DIR = Path("/kaggle/input/mallorn-astronomical-classification-challenge")

train_log = pd.read_csv(DATASET_DIR / "train_log.csv")
test_log = pd.read_csv(DATASET_DIR / "test_log.csv")

split_dirs = sorted([p for p in DATASET_DIR.glob("split_*") if p.is_dir()])

train_lc = pd.concat([pd.read_csv(d / "train_full_lightcurves.csv") for d in split_dirs], ignore_index=True)
test_lc = pd.concat([pd.read_csv(d / "test_full_lightcurves.csv") for d in split_dirs], ignore_index=True)

for df in (train_log, test_log):
    if "Z_err" in df.columns:
        df["Z_err"] = pd.to_numeric(df["Z_err"], errors="coerce")

## SECTION 3: PREPROCESSING

In [3]:
FILTERS = ["u", "g", "r", "i", "z", "y"]
f2i = {f: i for i, f in enumerate(FILTERS)}

def prep_lightcurves(df):
    df = df.copy()
    df["Filter"] = df["Filter"].astype(str).str.strip().str.lower()
    df["filter_id"] = df["Filter"].map(f2i).astype("int64")
    df["Time (MJD)"] = df["Time (MJD)"].astype("float64")
    df["Flux"] = df["Flux"].astype("float64")
    df["Flux_err"] = df["Flux_err"].astype("float64")
    return df

train_lc = prep_lightcurves(train_lc)
test_lc = prep_lightcurves(test_lc)

# # Fixed:
train_lc = train_lc.merge(train_log[["object_id", "EBV", "Z", "target"]], on="object_id", how="left")
test_lc = test_lc.merge(test_log[["object_id", "EBV", "Z"]], on="object_id", how="left")

print("Preprocessed lightcurves.")

Preprocessed lightcurves.


## SECTION 4: EXTINCTION CORRECTION

In [4]:
try:
    from extinction import fitzpatrick99
except ImportError:
    import subprocess
    subprocess.run(["pip", "-q", "install", "extinction==0.4.7"], check=True)
    from extinction import fitzpatrick99

EFF_WL = {
    "u": 3641.0, "g": 4704.0, "r": 6155.0,
    "i": 7504.0, "z": 8695.0, "y": 10056.0,
}

def apply_deextinction(df):
    df = df.copy()
    df["Flux_corr"] = df["Flux"].copy()
    df["Fluxerr_corr"] = df["Flux_err"].copy()
    
    for filt in df["Filter"].unique():
        if filt not in EFF_WL:
            continue
        filt_mask = df["Filter"] == filt
        wl_arr = np.array([EFF_WL[filt]])
        
        for ebv_val in df.loc[filt_mask, "EBV"].unique():
            if pd.isna(ebv_val) or ebv_val == 0:
                continue
            mask = filt_mask & (df["EBV"] == ebv_val)
            A_lambda = fitzpatrick99(wl_arr, ebv_val * 3.1)[0]
            factor = 10 ** (A_lambda / 2.5)
            df.loc[mask, "Flux_corr"] = df.loc[mask, "Flux"] * factor
            df.loc[mask, "Fluxerr_corr"] = df.loc[mask, "Flux_err"] * factor
    
    return df

print("Applying extinction correction...")
train_lc = apply_deextinction(train_lc)
test_lc = apply_deextinction(test_lc)
print("Done.")

def drop_bad_rows(lc, name=""):
    """Only drop rows with NaN/inf values. Keep negative flux."""
    before = len(lc)
    keep = (
        np.isfinite(lc["Time (MJD)"].values) &
        np.isfinite(lc["Flux_corr"].values) &
        np.isfinite(lc["Fluxerr_corr"].values)
    )
    lc = lc[keep].copy()
    print(f"{name}: Dropped {before - len(lc):,} bad rows (NaN/inf only)")
    return lc

train_lc = drop_bad_rows(train_lc, "Train")
test_lc = drop_bad_rows(test_lc, "Test")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.9/582.9 kB 9.9 MB/s eta 0:00:00
Applying extinction correction...
Done.
Train: Dropped 891 bad rows (NaN/inf only)
Test: Dropped 2,022 bad rows (NaN/inf only)


## SECTION 5: 3-SIGMA BASELINE DETECTION

In [5]:
def add_detection_columns_fast(lc, sigma_threshold=3.0):
    """Add detection columns - vectorized version (5-10x faster)."""
    lc = lc.copy()
    
    # Compute per-object statistics in one groupby pass
    def compute_group_stats(g):
        flux = g['Flux_corr'].values
        flux_err = g['Fluxerr_corr'].values
        n = len(flux)
        
        if n < 3:
            return pd.Series({'baseline': 0.0, 'noise': 1.0})
        
        baseline = np.percentile(flux, 25)
        noise_from_err = np.median(np.abs(flux_err))
        noise_from_mad = 1.4826 * np.median(np.abs(flux - np.median(flux)))
        noise = max(noise_from_err, noise_from_mad, 1e-10)
        
        return pd.Series({'baseline': baseline, 'noise': noise})
    
    # Single groupby operation
    stats = lc.groupby('object_id', sort=False).apply(compute_group_stats, include_groups=False)
    
    # Map stats back to original rows (fast)
    lc['baseline'] = lc['object_id'].map(stats['baseline'])
    lc['noise'] = lc['object_id'].map(stats['noise'])
    
    # Vectorized computation on full dataframe (very fast)
    lc['deviation'] = (lc['Flux_corr'] - lc['baseline']) / lc['noise']
    lc['is_detected'] = lc['deviation'] > sigma_threshold
    
    # Clean up
    lc.drop(columns=['noise'], inplace=True)
    
    return lc

print(f"\nComputing {SIGMA_THRESHOLD}-sigma detections...")
train_lc = add_detection_columns_fast(train_lc, SIGMA_THRESHOLD)
test_lc = add_detection_columns_fast(test_lc, SIGMA_THRESHOLD)

# Statistics
print(f"Train detection rate: {train_lc['is_detected'].mean():.1%}")
print(f"Test detection rate: {test_lc['is_detected'].mean():.1%}")




Computing 1.0-sigma detections...
Train detection rate: 36.3%
Test detection rate: 36.3%


## SECTION 6: FEATURE ENGINEERING

In [6]:
LSST_LAMBDA_NM = {'u': 367.0, 'g': 482.0, 'r': 622.0, 'i': 754.0, 'z': 869.0, 'y': 971.0}

if HAS_NUMBA:
    @jit(nopython=True, cache=True)
    def _safe_mad_jit(x):
        if len(x) == 0:
            return 0.0
        med = np.median(x)
        return float(np.median(np.abs(x - med)))
    
    @jit(nopython=True, cache=True)
    def _von_neumann_jit(x):
        n = len(x)
        if n < 3:
            return 0.0
        diff_sum = 0.0
        for i in range(n - 1):
            diff_sum += (x[i+1] - x[i]) ** 2
        mean = np.sum(x) / n
        var = np.sum((x - mean) ** 2) / (n - 1)
        return float(diff_sum / n) / (var + 1e-12)
    
    @jit(nopython=True, cache=True)  
    def _longest_decline_jit(flux):
        n = len(flux)
        if n < 2:
            return 0
        max_run = current_run = 0
        for i in range(n - 1):
            if flux[i+1] < flux[i]:
                current_run += 1
                if current_run > max_run:
                    max_run = current_run
            else:
                current_run = 0
        return max_run

def _safe_mad(x):
    if len(x) == 0:
        return 0.0
    if HAS_NUMBA:
        return _safe_mad_jit(np.asarray(x, dtype=np.float64))
    return float(np.median(np.abs(x - np.median(x))))

def _von_neumann_ratio(x):
    if len(x) < 3:
        return 0.0
    if HAS_NUMBA:
        return _von_neumann_jit(np.asarray(x, dtype=np.float64))
    dif = np.diff(x)
    v = np.var(x, ddof=1)
    return float(np.mean(dif**2) / (v + 1e-12))

def _powerlaw_decay_index(t, f, t_peak, is_detected, min_points=4):
    m = (t > t_peak) & (f > 0) & is_detected
    tt, ff = t[m] - t_peak, f[m]
    m2 = tt > 1e-3
    tt, ff = tt[m2], ff[m2]
    if len(tt) < min_points:
        return 0.0, 0.0
    try:
        slope, _, r, _, _ = stats.linregress(np.log(tt), np.log(ff))
        return float(np.clip(-slope, 0, 5)), float(r**2)
    except:
        return 0.0, 0.0

def _lin_slope(tt, yy):
    if len(tt) < 3 or (tt.max() - tt.min()) <= 0:
        return 0.0, 0.0
    s, _, r, _, _ = stats.linregress(tt, yy)
    return float(s), float(r**2)

def _longest_monotonic_run(flux, direction='decline'):
    if len(flux) < 2:
        return 0
    if direction == 'decline' and HAS_NUMBA:
        return _longest_decline_jit(np.asarray(flux, dtype=np.float64))
    diff = np.diff(flux) < 0 if direction == 'decline' else np.diff(flux) > 0
    max_run = current_run = 0
    for d in diff:
        if d:
            current_run += 1
            max_run = max(max_run, current_run)
        else:
            current_run = 0
    return int(max_run)

# Transient feature extraction
def find_transient_peak(obj_lc, z):
    result = {'has_transient': False, 't_peak': None, 'detections': None}
    all_det = []
    for filt in FILTERS:
        filt_lc = obj_lc[obj_lc['Filter'] == filt].copy()
        if len(filt_lc) < 3:
            continue
        flux = filt_lc['Flux_corr'].values
        flux_err = np.abs(filt_lc['Fluxerr_corr'].values) + 1e-10
        baseline = np.percentile(flux, 25)
        mad = 1.4826 * np.median(np.abs(flux - np.median(flux)))
        noise = max(mad, np.median(flux_err), 1e-10)
        snr = (flux - baseline) / noise
        rel_err = flux_err / (np.abs(flux) + 1e-10)
        good = (snr > 2.5) & (rel_err < 0.5) & (flux > 0)
        if good.sum() > 0:
            det = filt_lc[good].copy()
            det['filter'] = filt
            all_det.append(det)
    if not all_det:
        return result
    detections = pd.concat(all_det, ignore_index=True)
    if len(detections) < 4:
        return result
    for filt in ['r', 'g', 'i', 'z']:
        filt_det = detections[detections['filter'] == filt]
        if len(filt_det) > 0:
            peak_idx = filt_det['Flux_corr'].idxmax()
            result['t_peak'] = filt_det.loc[peak_idx, 'Time (MJD)']
            result['has_transient'] = True
            result['detections'] = detections
            break
    return result

def extract_transient_features(obj_lc, z):
    features = {'trans_delta_m15': np.nan, 'trans_delta_m50': np.nan, 'trans_fade_rate': np.nan}
    trans = find_transient_peak(obj_lc, z)
    if not trans['has_transient']:
        return features
    t_peak = trans['t_peak']
    detections = trans['detections']
    z_factor = 1 + z if z > 0 else 1.0
    
    for band in ['r', 'g', 'i', 'z']:
        band_det = detections[detections['filter'] == band]
        if len(band_det) < 3:
            continue
        times = band_det['Time (MJD)'].values
        fluxes = band_det['Flux_corr'].values
        flux_errs = np.abs(band_det['Fluxerr_corr'].values) + 1e-10
        t_rest = (times - t_peak) / z_factor
        valid = (t_rest >= -50) & (t_rest <= 300)
        if valid.sum() < 3:
            continue
        t_rest, fluxes, flux_errs = t_rest[valid], fluxes[valid], flux_errs[valid]
        try:
            from sklearn.gaussian_process import GaussianProcessRegressor
            from sklearn.gaussian_process.kernels import Matern, WhiteKernel
            t_mean, t_std = t_rest.mean(), max(t_rest.std(), 1)
            f_mean, f_std = fluxes.mean(), max(fluxes.std(), 1e-10)
            t_norm = (t_rest - t_mean) / t_std
            f_norm = (fluxes - f_mean) / f_std
            err_norm = flux_errs / f_std
            t_target = np.array([0, 15, 30, 50])
            t_target_norm = (t_target - t_mean) / t_std
            kernel = 1.0 * Matern(length_scale=1.0, nu=2.5) + WhiteKernel(0.1)
            gp = GaussianProcessRegressor(kernel=kernel, alpha=err_norm**2 + 0.01, 
                                          n_restarts_optimizer=1, random_state=42)
            gp.fit(t_norm.reshape(-1, 1), f_norm)
            f_pred = gp.predict(t_target_norm.reshape(-1, 1)) * f_std + f_mean
            if f_pred[0] > 0:
                mag_peak = -2.5 * np.log10(f_pred[0])
                if f_pred[1] > 0:
                    features['trans_delta_m15'] = -2.5 * np.log10(f_pred[1]) - mag_peak
                if f_pred[3] > 0:
                    features['trans_delta_m50'] = -2.5 * np.log10(f_pred[3]) - mag_peak
                mags, t_vals = [], []
                for j, t in enumerate([15, 30, 50]):
                    if f_pred[j+1] > 0:
                        mags.append(-2.5 * np.log10(f_pred[j+1]))
                        t_vals.append(t)
                if len(mags) >= 2:
                    features['trans_fade_rate'] = stats.linregress(t_vals, mags)[0]
            break
        except:
            continue
    return features

def compute_transient_features_all(lc_df, log_df, desc=""):
    z_lookup = log_df.set_index('object_id')['Z'].to_dict()
    results = []
    object_ids = log_df['object_id'].tolist()
    for i, oid in enumerate(object_ids):
        if i % 500 == 0 and i > 0:
            print(f"  {desc}: {i}/{len(object_ids)}")
        obj_lc = lc_df[lc_df['object_id'] == oid]
        feats = extract_transient_features(obj_lc, z_lookup.get(oid, 0))
        feats['object_id'] = oid
        results.append(feats)
    return pd.DataFrame(results)


In [7]:

def _extract_single_object(oid, g, z):
    """Extract features for a single object."""
    feat = {"object_id": oid}

    flux = g["Flux_corr"].values.astype(float)
    flux_err = g["Fluxerr_corr"].values.astype(float)
    time = g["Time (MJD)"].values.astype(float)
    filt = g["Filter"].values
    is_detected = g["is_detected"].values.astype(bool)
    deviation = g["deviation"].values.astype(float)
    n = len(g)

    feat["Z"] = z
    feat["n_obs"] = int(n)
    
    if n < 2:
        return feat

    band_data = {}
    for band in FILTERS:
        mask = filt == band
        if mask.sum() > 0:
            band_data[band] = {
                't': time[mask], 'f': flux[mask], 'f_err': flux_err[mask],
                'det': is_detected[mask], 'n': int(mask.sum())
            }
        else:
            band_data[band] = {'t': np.array([]), 'f': np.array([]), 
                               'f_err': np.array([]), 'det': np.array([], dtype=bool), 'n': 0}

    n_detected = int(np.sum(is_detected))
    feat["n_detected"] = n_detected
    feat["frac_detected"] = float(n_detected / n) if n > 0 else 0.0
    feat["max_deviation"] = float(np.max(deviation))
    feat["mean_deviation_detected"] = float(np.mean(deviation[is_detected])) if n_detected > 0 else 0.0

    def _cross_corr_fast(b1, b2, time_bins=15):
        d1, d2 = band_data[b1], band_data[b2]
        if d1['n'] < 3 or d2['n'] < 3:
            return 0.0
        t1, f1, t2, f2 = d1['t'], d1['f'], d2['t'], d2['f']
        t_min, t_max = min(t1.min(), t2.min()), max(t1.max(), t2.max())
        if t_max - t_min < 1:
            return 0.0
        bin_edges = np.linspace(t_min, t_max, time_bins + 1)
        f1_binned = np.full(time_bins, np.nan)
        f2_binned = np.full(time_bins, np.nan)
        bins1 = np.clip(np.digitize(t1, bin_edges) - 1, 0, time_bins - 1)
        bins2 = np.clip(np.digitize(t2, bin_edges) - 1, 0, time_bins - 1)
        for i in range(time_bins):
            m1, m2 = bins1 == i, bins2 == i
            if m1.sum() > 0: f1_binned[i] = np.median(f1[m1])
            if m2.sum() > 0: f2_binned[i] = np.median(f2[m2])
        valid = np.isfinite(f1_binned) & np.isfinite(f2_binned)
        if valid.sum() < 3:
            return 0.0
        x, y = f1_binned[valid], f2_binned[valid]
        x, y = x - x.mean(), y - y.mean()
        denom = np.sqrt(np.sum(x**2) * np.sum(y**2))
        return float(np.sum(x * y) / denom) if denom > 1e-12 else 0.0

    feat["cross_corr_gr"] = _cross_corr_fast('g', 'r')
    feat["cross_corr_gi"] = _cross_corr_fast('g', 'i')
    feat["cross_corr_ug"] = _cross_corr_fast('u', 'g')
    valid_corrs = [c for c in [feat["cross_corr_gr"], feat["cross_corr_gi"]] if c != 0]
    feat["cross_corr_min"] = float(np.min(valid_corrs)) if valid_corrs else 0.0

    n_negative = int(np.sum(flux < 0))
    n_positive = int(np.sum(flux > 0))
    feat["n_negative"] = n_negative
    feat["n_positive"] = n_positive
    flux_err_safe = np.abs(flux_err) + 1e-12
    feat["frac_significant_negative"] = float(np.mean(flux < -2 * flux_err_safe))
    feat["mean_negative_flux"] = float(np.mean(flux[flux < 0])) if n_negative > 0 else 0.0

    feat["longest_decline"] = _longest_monotonic_run(flux[is_detected], 'decline') if n_detected >= 3 else 0

    tmin, tmax = float(time.min()), float(time.max())
    span = float(tmax - tmin)
    feat["time_span"] = span
    dt = np.diff(time)
    feat["dt_med"] = float(np.median(dt)) if len(dt) else 0.0
    feat["dt_max"] = float(np.max(dt)) if len(dt) else 0.0
    feat["flux_mad"] = _safe_mad(flux)
    feat["flux_kurtosis"] = float(stats.kurtosis(flux)) if n > 3 else 0.0
    snr = flux / (np.abs(flux_err) + 1e-12)
    feat["snr_max"] = float(np.max(snr))
    feat["snr_min"] = float(np.min(snr))

    peak_idx = int(np.argmax(flux))
    t_peak, f_peak = float(time[peak_idx]), float(flux[peak_idx])
    feat["decay_time"] = float(tmax - t_peak) if peak_idx < n-1 else 0.0
    if f_peak > 0:
        m_half = flux >= 0.5 * f_peak
        feat["t_above_half"] = float(time[m_half].max() - time[m_half].min()) if np.any(m_half) else 0.0
    else:
        feat["t_above_half"] = 0.0
    feat["von_neumann"] = _von_neumann_ratio(flux)
    pre, post = time < t_peak, time > t_peak
    feat["slope_pre"], _ = _lin_slope(time[pre], flux[pre])
    feat["slope_post"], feat["r2_post"] = _lin_slope(time[post], flux[post])

    alpha, alpha_r2 = _powerlaw_decay_index(time, flux, t_peak, is_detected)
    feat["decay_alpha"] = alpha
    feat["decay_alpha_r2"] = alpha_r2
    feat["decay_alpha_deviation"] = float(abs(alpha - 5/3))
    feat["decay_is_tde_like"] = float(1.0 / (feat["decay_alpha_deviation"] + 0.1))
    
    def _decay_band(band):
        bd = band_data[band]
        if bd['n'] < 4: return 0.0, 0.0
        m = (bd['t'] > t_peak) & (bd['f'] > 0) & bd['det']
        if m.sum() < 4: return 0.0, 0.0
        t_post, f_post = bd['t'][m] - t_peak, bd['f'][m]
        m2 = t_post > 1.0
        if m2.sum() < 4: return 0.0, 0.0
        try:
            slope, _, r, _, _ = stats.linregress(np.log(t_post[m2]), np.log(f_post[m2]))
            return float(-slope), float(r**2)
        except: return 0.0, 0.0

    feat["decay_alpha_g"], feat["decay_r2_g"] = _decay_band("g")
    _, feat["decay_r2_r"] = _decay_band("r")
    _, feat["decay_r2_i"] = _decay_band("i")
    feat["decay_alpha_consistency"] = float(1.0 / (abs(feat["decay_alpha_g"] - alpha) + 0.1)) if feat["decay_alpha_g"] > 0 and alpha > 0 else 0.0

    # Peak prominence
    flux_med = np.median(flux)
    feat["peak_prominence"] = float(np.clip(f_peak / (flux_med + 1e-10), 0, 100)) if flux_med > 0 else float(np.clip(f_peak - flux_med, 0, 1e6))

    # Post-peak
    if np.sum(post) >= 6:
        f_post_arr, t_post_arr = flux[post], time[post]
        feat["post_mean"] = float(np.mean(f_post_arr))
        mid_t = 0.5 * (t_post_arr.min() + t_post_arr.max())
        late = t_post_arr > mid_t
        feat["late_slope"], _ = _lin_slope(t_post_arr[late], f_post_arr[late]) if np.sum(late) >= 3 else (0.0, 0.0)
    else:
        feat["post_mean"] = feat["late_slope"] = 0.0

    for band in FILTERS:
        bd = band_data[band]
        feat[f"n_{band}"] = bd['n']
        feat[f"n_detected_{band}"] = int(bd['det'].sum()) if bd['n'] > 0 else 0
        if bd['n'] >= 1:
            fb, tb = bd['f'], bd['t']
            feat[f"mean_{band}"] = float(np.mean(fb))
            feat[f"std_{band}"] = float(np.std(fb)) if bd['n'] > 1 else 0.0
            feat[f"p95_{band}"] = float(np.percentile(fb, 95)) if bd['n'] > 1 else float(fb[0])
            feat[f"p05_{band}"] = float(np.percentile(fb, 5)) if bd['n'] > 1 else float(fb[0])
            feat[f"amp_{band}"] = float(feat[f"p95_{band}"] - feat[f"p05_{band}"])
            ib = int(np.argmax(fb))
            feat[f"tpeak_{band}"], feat[f"fpeak_{band}"] = float(tb[ib]), float(fb[ib])
            feat[f"frac_negative_{band}"] = float(np.mean(fb < 0))
            feat[f"frac_detected_{band}"] = float(np.mean(bd['det']))
        else:
            for k in ["mean_", "std_", "p95_", "p05_", "amp_", "tpeak_", "fpeak_"]:
                feat[f"{k}{band}"] = 0.0
            feat[f"frac_negative_{band}"] = feat[f"frac_detected_{band}"] = 0.0

    feat["tlag_g_r"] = float(feat["tpeak_g"] - feat["tpeak_r"]) if feat["tpeak_g"] > 0 and feat["tpeak_r"] > 0 else 0.0

    def _color_fast(b1, b2):
        d1, d2 = band_data[b1], band_data[b2]
        if d1['n'] == 0 or d2['n'] == 0: return 0.0, 0.0, 0.0
        t1, f1, det1 = d1['t'], d1['f'], d1['det']
        t2, f2, det2 = d2['t'], d2['f'], d2['det']
        idx = np.clip(np.searchsorted(t2, t1), 0, len(t2)-1)
        idx_left = np.clip(idx - 1, 0, len(t2)-1)
        idx_best = np.where(np.abs(t2[idx_left] - t1) < np.abs(t2[idx] - t1), idx_left, idx)
        dt_best = np.abs(t2[idx_best] - t1)
        m = (dt_best <= 1.0) & (f1 > 0) & (f2[idx_best] > 0) & det1 & det2[idx_best]
        if np.sum(m) < 3: return 0.0, 0.0, 0.0
        t_pair, col = t1[m], -2.5 * np.log10(f1[m] / f2[idx_best][m])
        col_slope = stats.linregress(t_pair, col)[0] if len(col) >= 3 and (t_pair.max() - t_pair.min()) > 0 else 0.0
        return float(np.median(col)), float(np.std(col)), float(col_slope)

    _, feat["col_gr_std"], col_gr_slope = _color_fast("g", "r")
    feat["col_ug_med"], _, feat["col_ug_slope"] = _color_fast("u", "g")
    feat["col_ri_med"], _, _ = _color_fast("r", "i")
    feat["color_evolution_strength"] = float(abs(col_gr_slope))

    total_amp = feat["amp_u"] + feat["amp_g"] + feat["amp_r"] + feat["amp_i"] + feat["amp_z"] + feat["amp_y"] + 1e-10
    feat["frac_blue"] = float((feat["amp_u"] + feat["amp_g"]) / total_amp)
    feat["frac_red"] = float((feat["amp_i"] + feat["amp_z"] + feat["amp_y"]) / total_amp)
    feat["blue_red_amp_ratio"] = float((feat["amp_u"] + feat["amp_g"]) / (feat["amp_i"] + feat["amp_z"] + feat["amp_y"] + 1e-10))
    feat["fpeak_g_over_r"] = float(feat["fpeak_g"] / (feat["fpeak_r"] + 1e-10)) if feat["fpeak_r"] > 0 else 0.0
    feat["fpeak_u_over_r"] = float(feat["fpeak_u"] / (feat["fpeak_r"] + 1e-10)) if feat["fpeak_r"] > 0 else 0.0

    feat["smooth_decay"] = float((2 - feat["von_neumann"]) * feat["decay_alpha"])
    denom = feat["fpeak_r"] + feat["fpeak_i"] + feat["fpeak_z"]
    feat["temp_proxy"] = float((feat["fpeak_u"] + feat["fpeak_g"]) / (denom + 1e-10)) if denom > 0 else 0.0
    feat["smooth_decay_proxy"] = float(feat["r2_post"] * feat["decay_alpha_r2"])

    if z > 0:
        D_L = float(z * (1 + z/2))
        feat["L_abs_proxy"] = float(np.log10(max(f_peak * D_L**2 * (1 + z), 1e-10))) if f_peak > 0 else 0.0
    else:
        feat["L_abs_proxy"] = 0.0

    m = (time > t_peak) & (flux > 0) & is_detected
    tt, ff = time[m] - t_peak, flux[m]
    m2 = tt > 1e-3
    exp_r2 = 0.0
    if m2.sum() >= 4:
        try:
            _, _, r, _, _ = stats.linregress(tt[m2], np.log(ff[m2]))
            exp_r2 = float(r**2)
        except: pass
    feat["powerlaw_vs_exp_r2"] = float(alpha_r2 - exp_r2)
    feat["powerlaw_exp_r2_ratio"] = float((alpha_r2 + 0.01) / (exp_r2 + 0.01))
    
    return feat

def make_tde_features_fast(lc, log_df, n_jobs=4, use_parallel=True):
    """
    Extract TDE features - with optional parallel processing.
    
    Args:
        lc: Light curve DataFrame
        log_df: Metadata DataFrame
        n_jobs: Number of parallel workers (default 4)
        use_parallel: Whether to use parallel processing
    """
    import time as _time
    start = _time.time()
    
    lc = lc.sort_values(["object_id", "Time (MJD)"])
    z_lookup = log_df.set_index('object_id')['Z'].to_dict() if 'Z' in log_df.columns else {}
    
    # Pre-group data
    grouped = list(lc.groupby("object_id", sort=False))
    n_total = len(grouped)
    
    print(f"Extracting features for {n_total} objects...")
    
    if use_parallel and n_total > 100:
        # Parallel processing
                results = Parallel(n_jobs=n_jobs, verbose=5)(
            delayed(_extract_single_object)(oid, g, float(z_lookup.get(oid, 0.0)))
            for oid, g in grouped
        )
    else:
        # Sequential with progress
        results = []
        for i, (oid, g) in enumerate(grouped):
            if i % 500 == 0:
                print(f"  {i}/{n_total} objects processed...")
            results.append(_extract_single_object(oid, g, float(z_lookup.get(oid, 0.0))))
    
    elapsed = _time.time() - start
    print(f"Feature extraction completed in {elapsed:.1f}s ({n_total/elapsed:.1f} obj/s)")
    
    return pd.DataFrame(results)

def add_robust_coverage_features(df):
    """Add coverage features - only ones in TOP 100."""
    eps = 1e-9
    df["obs_density"] = df["n_obs"] / (df["time_span"] + 1.0)
    
    red_bands = ['i', 'z', 'y']
    n_det_cols = [f"n_detected_{b}" for b in red_bands if f"n_detected_{b}" in df.columns]
    df["red_detected"] = df[n_det_cols].sum(axis=1) if n_det_cols else 0
    
    frac_cols = [f"frac_detected_{b}" for b in red_bands if f"frac_detected_{b}" in df.columns]
    if frac_cols:
        df["frac_red_detected"] = df[frac_cols].mean(axis=1)
    elif "n_detected" in df.columns:
        df["frac_red_detected"] = df["red_detected"] / (df["n_detected"] + eps)
    else:
        df["frac_red_detected"] = 0.0
    
    return df


In [8]:
train_feats = make_tde_features_fast(train_lc, train_log, n_jobs=4)
test_feats = make_tde_features_fast(test_lc, test_log, n_jobs=4)

train_trans = compute_transient_features_all(train_lc, train_log, "Train")
test_trans = compute_transient_features_all(test_lc, test_log, "Test")

trans_cols = ['trans_delta_m15', 'trans_delta_m50', 'trans_fade_rate']
train_feats = train_feats.merge(train_trans[['object_id'] + trans_cols], on='object_id', how='left')
test_feats = test_feats.merge(test_trans[['object_id'] + trans_cols], on='object_id', how='left')

train_feats = add_robust_coverage_features(train_feats)
test_feats = add_robust_coverage_features(test_feats)
print(f"Features: train {train_feats.shape}, test {test_feats.shape}")

Extracting features for 3043 objects...


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:    7.8s
[Parallel(n_jobs=4)]: Done 212 tasks      | elapsed:    8.9s
[Parallel(n_jobs=4)]: Done 1652 tasks      | elapsed:   14.0s
[Parallel(n_jobs=4)]: Done 3043 out of 3043 | elapsed:   19.1s finished


Feature extraction completed in 19.6s (155.2 obj/s)
Extracting features for 7135 objects...


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  12 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done 632 tasks      | elapsed:    2.5s
[Parallel(n_jobs=4)]: Done 2072 tasks      | elapsed:    7.6s
[Parallel(n_jobs=4)]: Done 4088 tasks      | elapsed:   14.8s
[Parallel(n_jobs=4)]: Done 6680 tasks      | elapsed:   24.1s
[Parallel(n_jobs=4)]: Done 7135 out of 7135 | elapsed:   25.9s finished


Feature extraction completed in 27.2s (261.9 obj/s)
  Train: 500/3043
  Train: 1000/3043
  Train: 1500/3043
  Train: 2000/3043
  Train: 2500/3043
  Train: 3000/3043
  Test: 500/7135
  Test: 1000/7135
  Test: 1500/7135
  Test: 2000/7135
  Test: 2500/7135
  Test: 3000/7135
  Test: 3500/7135
  Test: 4000/7135
  Test: 4500/7135
  Test: 5000/7135
  Test: 5500/7135
  Test: 6000/7135
  Test: 6500/7135
  Test: 7000/7135
Features: train (3043, 130), test (7135, 130)


In [9]:
def compute_agn_features(lc, log_df):
    """
    Compute AGN features - only features in TOP 100.
    """
    lc = lc.sort_values(["object_id", "Time (MJD)"])
    rows = []
    
    for oid, g in lc.groupby("object_id"):
        feat = {"object_id": oid}
        
        flux = g["Flux_corr"].values
        time = g["Time (MJD)"].values
        is_detected = g["is_detected"].values if "is_detected" in g.columns else np.ones(len(flux), dtype=bool)
        
        n = len(flux)
        if n < 5:
            rows.append(feat)
            continue
        
        # Find peak
        peak_idx = np.argmax(flux)
        t_peak = time[peak_idx]
        f_peak = flux[peak_idx]
        
        pre_mask = time < t_peak
        n_pre = pre_mask.sum()
        
        if n_pre >= 3:
            flux_pre = flux[pre_mask]
            feat["agn_pre_range"] = float(np.max(flux_pre) - np.min(flux_pre))
        else:
            feat["agn_pre_range"] = 0.0
        
        post_mask = time > t_peak
        n_post = post_mask.sum()
        
        if n_post >= 5:
            flux_post = flux[post_mask]
            time_post = time[post_mask]
            
            # Count re-brightenings
            n_rebrighten = 0
            for i in range(1, len(flux_post)):
                if flux_post[i] > flux_post[i-1] * 1.1:
                    n_rebrighten += 1
            feat["agn_n_rebrighten"] = int(n_rebrighten)
            feat["agn_rebrighten_rate"] = float(n_rebrighten / (n_post - 1)) if n_post > 1 else 0.0
            
            # Monotonicity
            if len(flux_post) >= 2:
                declines = np.diff(flux_post) < 0
                feat["agn_post_monotonicity"] = float(np.mean(declines))
            else:
                feat["agn_post_monotonicity"] = 0.0
            
            # Late-time behavior
            late_mask = time_post > (t_peak + 0.7 * (time_post.max() - t_peak))
            if late_mask.sum() >= 2:
                flux_late = flux_post[late_mask]
                feat["agn_late_std"] = float(np.std(flux_late))
                feat["agn_late_to_peak"] = float(np.mean(flux_late) / (f_peak + 1e-10))
            else:
                feat["agn_late_std"] = 0.0
                feat["agn_late_to_peak"] = 0.0
        else:
            feat["agn_n_rebrighten"] = 0
            feat["agn_rebrighten_rate"] = 0.0
            feat["agn_post_monotonicity"] = 0.0
            feat["agn_late_std"] = 0.0
            feat["agn_late_to_peak"] = 0.0
        
        if n >= 5:
            lags = [1, 5, 20]
            for lag in lags:
                if n > lag:
                    sf = np.mean((flux[lag:] - flux[:-lag])**2)
                    feat[f"agn_sf_lag{lag}"] = float(np.sqrt(sf))
                else:
                    feat[f"agn_sf_lag{lag}"] = 0.0
            
            if feat["agn_sf_lag1"] > 0:
                # Need lag10 for ratio
                if n > 10:
                    sf10 = np.mean((flux[10:] - flux[:-10])**2)
                    feat["agn_sf_lag10"] = float(np.sqrt(sf10))
                else:
                    feat["agn_sf_lag10"] = 0.0
                feat["agn_sf_ratio_10_1"] = float(feat["agn_sf_lag10"] / (feat["agn_sf_lag1"] + 1e-10))
            else:
                feat["agn_sf_ratio_10_1"] = 0.0
        else:
            for lag in [1, 5, 20]:
                feat[f"agn_sf_lag{lag}"] = 0.0
            feat["agn_sf_ratio_10_1"] = 0.0
        
        rows.append(feat)
    
    return pd.DataFrame(rows)

# Compute AGN features
agn_train = compute_agn_features(train_lc, train_log)
agn_test = compute_agn_features(test_lc, test_log)

# Merge with existing features
train_feats = train_feats.merge(agn_train, on='object_id', how='left')
test_feats = test_feats.merge(agn_test, on='object_id', how='left')



In [10]:
if SNCOSMO_TRAIN_PATH.exists() and SNCOSMO_TEST_PATH.exists():
    sncosmo_train = pd.read_parquet(SNCOSMO_TRAIN_PATH)
    sncosmo_test = pd.read_parquet(SNCOSMO_TEST_PATH)
    print(f"Loaded train sncosmo: {sncosmo_train.shape}")
    print(f"Loaded test sncosmo: {sncosmo_test.shape}")
    
    # Select only TOP 100 features
    sncosmo_key_cols = [
        'object_id',
        'sn_salt2_rchisq', 'sn_salt3_rchisq', 
        'sn_best_ia_rchisq', 'sn_is_good_ia_fit',  # Keep is_good_ia_fit for veto logic
        'sn_salt2_x1',
        'sn_best_cc_rchisq',
    ]
    sncosmo_key_cols = [c for c in sncosmo_key_cols if c in sncosmo_train.columns]
    
    # Merge with existing features
    n_before = len(train_feats.columns)
    train_feats = train_feats.merge(sncosmo_train[sncosmo_key_cols], on='object_id', how='left')
    test_feats = test_feats.merge(sncosmo_test[sncosmo_key_cols], on='object_id', how='left')
    print(f"Added {len(train_feats.columns) - n_before} sncosmo features")
else:
    print("WARNING: SNCOSMO features not found!")


Loaded train sncosmo: (3043, 44)
Loaded test sncosmo: (7135, 44)
Added 6 sncosmo features


In [11]:

import time as _time

# CONFIGURATION
GP_CACHE_DIR = Path("/kaggle/input/gpfeatures/gp_cache/")
GP_CACHE_DIR.mkdir(exist_ok=True)

# Cache file paths
TRAIN_GP_CACHE = GP_CACHE_DIR / "train_gp_features_v11.parquet"
TEST_GP_CACHE = GP_CACHE_DIR / "test_gp_features_v11.parquet"

# Control flags
USE_GP_CACHE = True   # Set False to force recomputation
SAVE_GP_CACHE = True  # Set False to skip saving (for testing)

# GP hyperparameters
GP_LENGTH_SCALE = 20  # days (observer frame)
MIN_GP_POINTS = 5     # minimum points to fit GP

# GP HELPER FUNCTIONS (only defined if not already present)
if 'fit_gp_single_band_v11' not in dir():
    from sklearn.gaussian_process import GaussianProcessRegressor
    from sklearn.gaussian_process.kernels import Matern, ConstantKernel, WhiteKernel
    
    def flux_to_abmag_v11(flux_ujy):
        """Convert flux in μJy to AB magnitude."""
        flux_ujy = np.asarray(flux_ujy, dtype=float)
        scalar_input = flux_ujy.ndim == 0
        flux_ujy = np.atleast_1d(flux_ujy)
        result = np.full_like(flux_ujy, np.nan, dtype=float)
        valid = flux_ujy > 0
        result[valid] = -2.5 * np.log10(flux_ujy[valid] * 1e-6) + 8.90
        if scalar_input:
            return float(result[0])
        return result

    def fit_gp_single_band_v11(t_obs, f, f_err):
        """Fit GP to a single band's light curve."""
        if len(t_obs) < MIN_GP_POINTS:
            return None
        kernel = (
            ConstantKernel(1.0, (0.01, 100)) * 
            Matern(length_scale=GP_LENGTH_SCALE, length_scale_bounds=(5, 200), nu=2.5) +
            WhiteKernel(noise_level=0.1, noise_level_bounds=(0.001, 10))
        )
        gp = GaussianProcessRegressor(
            kernel=kernel, alpha=f_err**2 + 1e-6,
            n_restarts_optimizer=2, normalize_y=True
        )
        try:
            gp.fit(t_obs.reshape(-1, 1), f)
            return gp
        except:
            return None

    def compute_decay_slope_v11(t_rest, f, start_day, end_day):
        """Compute linear decay slope in rest-frame window."""
        mask = (t_rest >= start_day) & (t_rest <= end_day)
        if mask.sum() < 3:
            return np.nan, np.nan
        t_fit, f_fit = t_rest[mask], f[mask]
        if np.std(f_fit) < 1e-10:
            return np.nan, np.nan
        try:
            slope, _, r_value, _, _ = stats.linregress(t_fit, f_fit)
            return slope, r_value**2
        except:
            return np.nan, np.nan

    def compute_time_above_v11(t_rest, f, f_peak, threshold_frac):
        """Time that flux stays above threshold (rest frame)."""
        if f_peak <= 0:
            return 0.0
        above = f > threshold_frac * f_peak
        if above.sum() == 0:
            return 0.0
        t_above = t_rest[above]
        return t_above.max() - t_above.min()

    print("  GP helper functions defined")

# MAIN GP FEATURE EXTRACTION FUNCTION
def extract_gp_features_single_v11(object_id, lc_df, z):
    """
    Extract GP features for a single object.
    All time-based features computed in REST FRAME.
    
    Returns: dict of features (without object_id)
    """
    obj_lc = lc_df[lc_df['object_id'] == object_id]
    
    if len(obj_lc) < 10:
        return {}
    
    # Rest-frame factor: t_rest = t_obs / (1 + z)
    z_factor = 1 + z if z > 0 else 1.0
    
    features = {}
    
    # Create time grid for GP predictions
    all_t = obj_lc['Time (MJD)'].values
    t_min, t_max = all_t.min(), all_t.max()
    t_grid = np.linspace(t_min, t_max, 500)
    
    # Fit GP to each band
    band_data = {}
    for band in ['u', 'g', 'r', 'i', 'z', 'y']:
        bd = obj_lc[obj_lc['Filter'] == band].sort_values('Time (MJD)')
        if len(bd) < MIN_GP_POINTS:
            continue
        t = bd['Time (MJD)'].values
        f = bd['Flux_corr'].values
        f_err = np.abs(bd['Fluxerr_corr'].values) + 1e-6
        
        gp = fit_gp_single_band_v11(t, f, f_err)
        if gp is None:
            continue
        
        f_pred, _ = gp.predict(t_grid.reshape(-1, 1), return_std=True)
        band_data[band] = {
            'f_pred': f_pred,
            't_peak_obs': t_grid[np.argmax(f_pred)],
            'f_peak': f_pred.max()
        }
    
    if len(band_data) == 0:
        return features
    
    # Create pseudo-bolometric light curve
    f_bol = np.zeros_like(t_grid)
    for bd in band_data.values():
        f_bol += np.clip(bd['f_pred'], 0, None)  # Only positive flux
    
    # Find bolometric peak
    bol_peak_idx = np.argmax(f_bol)
    t_peak_obs = t_grid[bol_peak_idx]
    f_peak_bol = f_bol[bol_peak_idx]
    
    # Convert to REST FRAME relative to peak
    # t_rest = 0 at peak, negative before, positive after
    t_rest = (t_grid - t_peak_obs) / z_factor
    
    # ---------------------------------------------
    # BOLOMETRIC FEATURES
    # ---------------------------------------------
    features['gp_f_peak_bol'] = f_peak_bol
    
    # Amplitude ratio (peak / baseline)
    bol_baseline = np.percentile(f_bol, 10)
    features['gp_amplitude_ratio'] = f_peak_bol / max(bol_baseline, 0.1)
    
    # Decay slopes at different windows (REST FRAME)
    for window in [30, 60, 100]:
        slope, r2 = compute_decay_slope_v11(t_rest, f_bol, 5, window)
        features[f'gp_decay_slope_{window}d'] = slope
        features[f'gp_decay_r2_{window}d'] = r2
        
        # Normalized by peak (dimensionless decay rate)
        if not np.isnan(slope) and f_peak_bol > 0:
            features[f'gp_decay_rate_norm_{window}d'] = slope / f_peak_bol
        else:
            features[f'gp_decay_rate_norm_{window}d'] = np.nan
    
    # Time above thresholds (REST FRAME)
    features['gp_t_above_half'] = compute_time_above_v11(t_rest, f_bol, f_peak_bol, 0.5)
    features['gp_t_above_10pct'] = compute_time_above_v11(t_rest, f_bol, f_peak_bol, 0.1)
    
    # ---------------------------------------------
    # PER-BAND DECAY SLOPES
    # ---------------------------------------------
    for band in ['g', 'r', 'i', 'z']:
        if band not in band_data:
            features[f'gp_decay_slope_{band}'] = np.nan
            features[f'gp_decay_rate_norm_{band}'] = np.nan
            continue
        
        bd = band_data[band]
        # Use same t_rest (relative to BOLOMETRIC peak)
        slope, _ = compute_decay_slope_v11(t_rest, bd['f_pred'], 5, 60)
        features[f'gp_decay_slope_{band}'] = slope
        
        if not np.isnan(slope) and bd['f_peak'] > 0:
            features[f'gp_decay_rate_norm_{band}'] = slope / bd['f_peak']
        else:
            features[f'gp_decay_rate_norm_{band}'] = np.nan
    
    # ---------------------------------------------
    # COLOR EVOLUTION (g-r)
    # ---------------------------------------------
    if 'g' in band_data and 'r' in band_data:
        g_pred = band_data['g']['f_pred']
        r_pred = band_data['r']['f_pred']
        
        # At peak (t_rest ~ 0)
        idx_peak = np.argmin(np.abs(t_rest - 0))
        g_peak, r_peak = g_pred[idx_peak], r_pred[idx_peak]
        
        # At 30d rest frame
        idx_30d = np.argmin(np.abs(t_rest - 30))
        g_30d, r_30d = g_pred[idx_30d], r_pred[idx_30d]
        
        if g_peak > 0 and r_peak > 0:
            features['gp_color_gr_peak'] = flux_to_abmag_v11(g_peak) - flux_to_abmag_v11(r_peak)
        else:
            features['gp_color_gr_peak'] = np.nan
        
        if g_30d > 0 and r_30d > 0:
            features['gp_color_gr_30d'] = flux_to_abmag_v11(g_30d) - flux_to_abmag_v11(r_30d)
        else:
            features['gp_color_gr_30d'] = np.nan
        
        # Color evolution: positive = getting redder, negative = getting bluer
        if not np.isnan(features.get('gp_color_gr_peak', np.nan)) and \
           not np.isnan(features.get('gp_color_gr_30d', np.nan)):
            features['gp_color_evolution_gr'] = features['gp_color_gr_30d'] - features['gp_color_gr_peak']
        else:
            features['gp_color_evolution_gr'] = np.nan
    else:
        features['gp_color_gr_peak'] = np.nan
        features['gp_color_gr_30d'] = np.nan
        features['gp_color_evolution_gr'] = np.nan
    
    return features

def compute_gp_features_all(lc_df, log_df, cache_path, desc=""):
    """
    Compute GP features for all objects with progress tracking and caching.
    
    Args:
        lc_df: Light curve DataFrame (must have Flux_corr, Fluxerr_corr columns)
        log_df: Metadata DataFrame (must have object_id, Z columns)
        cache_path: Path to save/load parquet cache
        desc: Description for progress messages
    
    Returns:
        DataFrame with object_id and GP features
    """
    # Check cache first
    if USE_GP_CACHE and cache_path.exists():
        print(f"  Loading cached {desc} GP features from {cache_path}")
        df = pd.read_parquet(cache_path)
        print(f"  Loaded: {len(df)} objects, {len(df.columns)-1} features")
        return df
    
    # Compute features
    print(f"  Computing {desc} GP features (this takes ~10-15 min)...")
    z_lookup = log_df.set_index('object_id')['Z'].to_dict()
    object_ids = log_df['object_id'].tolist()
    
    results = []
    start_time = _time.time()
    
    for i, oid in enumerate(object_ids):
        if i % 200 == 0:
            elapsed = _time.time() - start_time
            rate = (i + 1) / elapsed if elapsed > 0 else 0
            eta = (len(object_ids) - i) / rate if rate > 0 else 0
            print(f"    {desc}: {i}/{len(object_ids)} ({rate:.1f}/s, ETA: {eta/60:.1f}min)")
        
        z = z_lookup.get(oid, 0)
        if pd.isna(z):
            z = 0
        
        feats = extract_gp_features_single_v11(oid, lc_df, z)
        feats['object_id'] = oid
        results.append(feats)
    
    elapsed = _time.time() - start_time
    print(f"    {desc}: Done! {len(object_ids)} objects in {elapsed/60:.1f} min")
    
    df = pd.DataFrame(results)
    
    # Reorder columns: object_id first
    cols = ['object_id'] + [c for c in df.columns if c != 'object_id']
    df = df[cols]
    
    # Save cache
    if SAVE_GP_CACHE:
        df.to_parquet(cache_path, index=False)
        print(f"    Saved to {cache_path}")
    
    return df

# COMPUTE OR LOAD GP FEATURES
print("\nComputing/loading GP features...")

train_gp = compute_gp_features_all(train_lc, train_log, TRAIN_GP_CACHE, "Train")
test_gp = compute_gp_features_all(test_lc, test_log, TEST_GP_CACHE, "Test")

print(f"\nTrain GP: {train_gp.shape}, Test GP: {test_gp.shape}")

# DEFINE WHICH GP FEATURES TO USE (most significant from FP/FN analysis)
GP_FEATURES_TO_USE = [
    # Features in TOP 100 only
    'gp_decay_slope_30d',
    'gp_decay_slope_60d', 
    'gp_decay_slope_r',
    'gp_decay_r2_100d',
    'gp_f_peak_bol',
    'gp_t_above_half',
    'gp_color_evolution_gr',
]

# Filter to only features that exist
gp_cols_available = [c for c in GP_FEATURES_TO_USE if c in train_gp.columns]
print(f"\nGP features to merge: {len(gp_cols_available)}")

# MERGE WITH EXISTING FEATURES
print("\nMerging GP features with existing features...")

# Check for column name conflicts
existing_cols = set(train_feats.columns)
new_cols = set(gp_cols_available)
overlap = existing_cols & new_cols

if overlap:
    print(f"  WARNING: Dropping overlapping columns from existing: {overlap}")
    train_feats = train_feats.drop(columns=list(overlap), errors='ignore')
    test_feats = test_feats.drop(columns=list(overlap), errors='ignore')

# Merge
merge_cols = ['object_id'] + gp_cols_available
train_feats = train_feats.merge(train_gp[merge_cols], on='object_id', how='left')
test_feats = test_feats.merge(test_gp[merge_cols], on='object_id', how='left')

print(f"  Train features: {train_feats.shape}")
print(f"  Test features: {test_feats.shape}")

# Show GP feature coverage
print("\nGP feature coverage (train):")
for col in gp_cols_available[:6]:
    cov = train_feats[col].notna().mean() * 100
    mean_val = train_feats[col].mean()
    print(f"  {col}: {cov:.1f}% coverage, mean={mean_val:.4f}")
print("  ...")

# Quick validation: compare TDE vs non-TDE
if 'target' in train_log.columns:
    print("\nQuick validation (TDE vs non-TDE):")
    train_with_target = train_feats.merge(train_log[['object_id', 'target']], on='object_id')
    tde = train_with_target[train_with_target['target'] == 1]
    non_tde = train_with_target[train_with_target['target'] == 0]
    
    for col in ['gp_decay_slope_60d', 'gp_t_above_half', 'gp_color_evolution_gr']:
        if col in train_feats.columns:
            tde_mean = tde[col].mean()
            non_mean = non_tde[col].mean()
            print(f"  {col}: TDE={tde_mean:.4f}, non-TDE={non_mean:.4f}")

print("GP features integrated successfully!")

  GP helper functions defined

Computing/loading GP features...
  Loading cached Train GP features from /kaggle/input/gpfeatures/gp_cache/train_gp_features_v11.parquet
  Loaded: 3043 objects, 24 features
  Loading cached Test GP features from /kaggle/input/gpfeatures/gp_cache/test_gp_features_v11.parquet
  Loaded: 7135 objects, 24 features

Train GP: (3043, 25), Test GP: (7135, 25)

GP features to merge: 7

Merging GP features with existing features...
  Train features: (3043, 154)
  Test features: (7135, 154)

GP feature coverage (train):
  gp_decay_slope_30d: 98.2% coverage, mean=-0.1810
  gp_decay_slope_60d: 98.2% coverage, mean=-0.1311
  gp_decay_slope_r: 98.1% coverage, mean=-0.0408
  gp_decay_r2_100d: 98.2% coverage, mean=0.8770
  gp_f_peak_bol: 100.0% coverage, mean=17.0451
  gp_t_above_half: 100.0% coverage, mean=361.4974
  ...

Quick validation (TDE vs non-TDE):
  gp_decay_slope_60d: TDE=-0.1605, non-TDE=-0.1296
  gp_t_above_half: TDE=146.4873, non-TDE=372.4931
  gp_color_evol

In [32]:
# Validate features
# Check which selected features are in train_feats
# Note: gru_pred and transformer_pred come from sequence models later
SELECTED_FEATURES = train_feats.columns
# features_before_seq = [f for f in SELECTED_FEATURES if f not in ['gru_pred', 'transformer_pred']]

# present = [f for f in features_before_seq if f in train_feats.columns]
# missing = [f for f in features_before_seq if f not in train_feats.columns]

# print(f"\nPresent: {len(present)}/{len(features_before_seq)} features")
# if missing:
#     print(f"\nMISSING FEATURES ({len(missing)}):")
#     for m in missing:
#         print(f"  - {m}")
# else:
#     print("\nAll non-sequence features present!")

print(f"\nTotal columns in train_feats: {len(train_feats.columns)}")



Total columns in train_feats: 156


## SECTION 7: SEQUENCE DATA PREPARATION

In [14]:
def build_enhanced_sequences(lc):
    """Build sequences with enhanced features including detection info."""
    lc = lc.sort_values(["object_id", "Time (MJD)"]).copy()
    
    seqs = {}
    for oid, g in lc.groupby("object_id"):
        flux = g["Flux_corr"].values.astype("float32")
        flux_err = g["Fluxerr_corr"].values.astype("float32")
        time = g["Time (MJD)"].values.astype("float32")
        fid = g["filter_id"].values.astype("int64")
        is_detected = g["is_detected"].values.astype("float32")
        deviation = g["deviation"].values.astype("float32")

         # Get redshift for this object
        z = g["Z"].iloc[0]  # Same for all rows of this object
        
        # Convert observer-frame time to rest-frame time
        dt_obs = np.diff(time, prepend=time[0]).astype("float32")
        dt_obs[0] = 0
        dt = dt_obs# / (1 + z)

        
        flux_mean = flux.mean()
        flux_std = flux.std() + 1e-6
       # flux_norm = ((flux - flux_mean) / flux_std).astype("float32")
        snr = (flux / (np.abs(flux_err) + 1e-6)).astype("float32")
        log_err = np.log(np.abs(flux_err) + 1e-6).astype("float32")
        
        flux_norm_raw = ((flux - flux_mean) / flux_std).astype("float32")
        snr_weight = np.clip(np.abs(snr) / 3.0, 0, 1).astype("float32")
        flux_norm = (flux_norm_raw * snr_weight).astype("float32")
        
        flux_diff = np.diff(flux, prepend=flux[0]).astype("float32")
        flux_diff[0] = 0
        

        
        seqs[oid] = {
            "dt": dt, "filter_id": fid, "flux": flux, "flux_err": flux_err,
            "flux_norm": flux_norm, "flux_diff": flux_diff, "snr": snr, "log_err": log_err,
            "is_detected": is_detected, "deviation": deviation,
        }
    
    return seqs

print("Building sequences...")
train_seqs = build_enhanced_sequences(train_lc)
test_seqs = build_enhanced_sequences(test_lc)
print(f"Train sequences: {len(train_seqs)}, Test sequences: {len(test_seqs)}")

Building sequences...
Train sequences: 3043, Test sequences: 7135


## SECTION 8: GRU MODEL (v2 architecture with detection input)

In [15]:
class EnhancedSeqDataset(Dataset):
    def __init__(self, ids, seqs, y=None,
                 cad_drop_max=0.50, cad_drop_p=0.60, cad_drop_min=0.10,
                 band_drop_p=0.30, band_drop_bias_blue=0.70, min_keep=10):
        self.ids = ids
        self.seqs = seqs
        self.y = y
        self.cad_drop_max = float(cad_drop_max)
        self.cad_drop_p = float(cad_drop_p)
        self.cad_drop_min = float(cad_drop_min)
        self.band_drop_p = float(band_drop_p)
        self.band_drop_bias_blue = float(band_drop_bias_blue)
        self.min_keep = int(min_keep)

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, idx):
        oid = self.ids[idx]
        s = self.seqs[oid]

        # Add is_detected and deviation to input features
        x_num = np.stack([
            s["flux_norm"], s["flux_diff"], s["snr"], s["log_err"], s["dt"],
            s["is_detected"], s["deviation"]
        ], axis=1).astype(np.float32)
        x_fid = s["filter_id"].astype(np.int64)
        T = len(x_fid)

        if self.y is not None and T > self.min_keep:
            if np.random.rand() < self.band_drop_p:
                if np.random.rand() < self.band_drop_bias_blue:
                    drop_band = np.random.choice([f2i["u"], f2i["g"]])
                else:
                    drop_band = np.random.choice([f2i[f] for f in FILTERS])
                m = (x_fid != drop_band)
                if int(m.sum()) >= self.min_keep:
                    x_num, x_fid = x_num[m], x_fid[m]
                    T = len(x_fid)

            if (self.cad_drop_max > 0) and (np.random.rand() < self.cad_drop_p) and (T > self.min_keep):
                frac = np.random.uniform(self.cad_drop_min, self.cad_drop_max)
                keep = max(self.min_keep, int(round((1.0 - frac) * T)))
                if keep < T:
                    idx_keep = np.sort(np.random.choice(T, size=keep, replace=False))
                    x_num, x_fid = x_num[idx_keep], x_fid[idx_keep]

        if self.y is None:
            return oid, x_num, x_fid
        return oid, x_num, x_fid, float(self.y[idx])

def collate_batch(batch):
    has_y = (len(batch[0]) == 4)
    oids = [b[0] for b in batch]
    lens = [len(b[2]) for b in batch]
    T = max(lens)
    n_features = batch[0][1].shape[1]

    x_num = torch.zeros(len(batch), T, n_features, dtype=torch.float32)
    x_fid = torch.zeros(len(batch), T, dtype=torch.long)
    mask = torch.zeros(len(batch), T, dtype=torch.float32)

    for i, b in enumerate(batch):
        L = len(b[2])
        x_num[i, :L] = torch.from_numpy(b[1])
        x_fid[i, :L] = torch.from_numpy(b[2])
        mask[i, :L] = 1.0

    if has_y:
        y = torch.tensor([b[3] for b in batch], dtype=torch.float32)
        return oids, x_num, x_fid, mask, y
    return oids, x_num, x_fid, mask

class AttentionPooling(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.attention = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.Tanh(),
            nn.Linear(hidden_dim // 2, 1)
        )
    
    def forward(self, x, mask):
        scores = self.attention(x).squeeze(-1)
        scores = scores.masked_fill(mask == 0, -1e9)
        weights = F.softmax(scores, dim=1)
        pooled = (x * weights.unsqueeze(-1)).sum(dim=1)
        return pooled, weights

class AttentionGRUEncoder(nn.Module):
    """v2 architecture with detection input."""
    def __init__(self, n_filters=6, filt_emb=8, hidden=96, num_in=7, dropout=0.4, n_layers=2):
        super().__init__()
        self.femb = nn.Embedding(n_filters, filt_emb)
        self.proj = nn.Sequential(
            nn.Linear(num_in + filt_emb, hidden),
            nn.LayerNorm(hidden),
            nn.ReLU(),
            nn.Dropout(dropout)
        )
        self.gru = nn.GRU(hidden, hidden, num_layers=n_layers, batch_first=True,
                         bidirectional=True, dropout=dropout if n_layers > 1 else 0)
        self.ln = nn.LayerNorm(hidden * 2)
        self.drop = nn.Dropout(dropout)
        self.attention_pool = AttentionPooling(hidden * 2)
        self.head = nn.Sequential(
            nn.Linear(hidden * 2, hidden),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden, 1)
        )
    
    def forward(self, x_num, x_fid, mask):
        e = self.femb(x_fid)
        x = torch.cat([x_num, e], dim=-1)
        x = self.proj(x)
        x, _ = self.gru(x)
        x = self.ln(x)
        x = self.drop(x)
        emb, _ = self.attention_pool(x, mask)
        logit = self.head(emb).squeeze(-1)
        return logit, emb

## SECTION 9: TRANSFORMER MODEL (NEW in v9!)

In [16]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000, dropout=0.1):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
    
    def forward(self, x):
        x = x + self.pe[:, :x.size(1)]
        return self.dropout(x)

class TransformerEncoder(nn.Module):
    """Transformer-based encoder for light curves."""
    def __init__(self, n_filters=6, filt_emb=8, hidden=96, num_in=7, 
                 n_heads=4, n_layers=2, dropout=0.3):
        super().__init__()
        self.hidden = hidden
        
        self.femb = nn.Embedding(n_filters, filt_emb)
        self.input_proj = nn.Linear(num_in + filt_emb, hidden)
        
        self.pos_encoder = PositionalEncoding(hidden, dropout=dropout)
        
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=hidden, nhead=n_heads, dim_feedforward=hidden*4,
            dropout=dropout, activation='gelu', batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=n_layers)
        
        self.attention_pool = AttentionPooling(hidden)
        
        self.head = nn.Sequential(
            nn.Linear(hidden, hidden),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden, 1)
        )
    
    def forward(self, x_num, x_fid, mask):
        e = self.femb(x_fid)
        x = torch.cat([x_num, e], dim=-1)
        x = self.input_proj(x)
        x = self.pos_encoder(x)
        
        # Transformer expects mask where True = ignore
        src_key_padding_mask = (mask == 0)
        x = self.transformer(x, src_key_padding_mask=src_key_padding_mask)
        
        emb, _ = self.attention_pool(x, mask)
        logit = self.head(emb).squeeze(-1)
        return logit, emb

## SECTION 10: K-FOLD OOF TRAINING FOR SEQUENCE MODELS

In [17]:
def save_seq_cache(name, oof, test, aucs):
    '''Save sequence model results to cache.'''
    cache_path = CACHE_DIR / f"{name.lower()}_cache.pt"
    torch.save({
        'oof': oof,
        'test': test,
        'aucs': aucs,
        'timestamp': pd.Timestamp.now().isoformat(),
    }, cache_path)
    print(f"  Saved {name} cache to {cache_path}")

def load_seq_cache(name, y):
    '''Load sequence model results from cache.'''
    cache_path = CACHE_DIR / f"{name.lower()}_cache.pt"
    if not cache_path.exists():
        print(f"  Cache not found: {cache_path}")
        return None
    cache = torch.load(cache_path, map_location='cpu',weights_only=False)
    oof_auc = roc_auc_score(y, cache['oof'])
    print(f"  Loaded {name} cache (OOF AUC: {oof_auc:.4f}, from {cache.get('timestamp', 'unknown')})")
    return cache

def train_seq_model_fold(model_class, model_params, train_ids, train_seqs, y_train,
                         val_ids, y_val, epochs=100, lr=1e-3, patience=30, device=DEVICE):
    """Generic training for GRU or Transformer."""
    ds_tr = EnhancedSeqDataset(train_ids, train_seqs, y=y_train,
                               cad_drop_max=0.50, cad_drop_p=0.60, cad_drop_min=0.10,
                               band_drop_p=0.30, band_drop_bias_blue=0.70, min_keep=10)
    ds_va = EnhancedSeqDataset(val_ids, train_seqs, y=y_val,
                               cad_drop_max=0.0, cad_drop_p=0.0, band_drop_p=0.0, min_keep=10)
    
    dl_tr = DataLoader(ds_tr, batch_size=128, shuffle=True, collate_fn=collate_batch, num_workers=0)
    dl_va = DataLoader(ds_va, batch_size=256, shuffle=False, collate_fn=collate_batch, num_workers=0)
    
    model = model_class(**model_params).to(device)
    opt = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=0.03)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=epochs, eta_min=1e-6)
    
    pos = y_train.sum()
    neg = len(y_train) - pos
    pos_weight = torch.tensor([neg / (pos + 1e-6)], device=device)
    crit = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
    
    best_auc, best_state, no_improve = 0.0, None, 0
    
    for ep in range(1, epochs + 1):
        model.train()
        for batch in dl_tr:
            oids, x_num, x_fid, mask, yb = batch
            x_num, x_fid, mask, yb = x_num.to(device), x_fid.to(device), mask.to(device), yb.to(device)
            opt.zero_grad()
            logits, _ = model(x_num, x_fid, mask)
            loss = crit(logits, yb)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            opt.step()
        scheduler.step()
        
        model.eval()
        val_preds = []
        with torch.no_grad():
            for batch in dl_va:
                oids, x_num, x_fid, mask, yb = batch
                x_num, x_fid, mask = x_num.to(device), x_fid.to(device), mask.to(device)
                logits, _ = model(x_num, x_fid, mask)
                val_preds.append(torch.sigmoid(logits).cpu().numpy())
        
        val_preds = np.concatenate(val_preds)
        auc = roc_auc_score(y_val, val_preds)
        
        if auc > best_auc:
            best_auc = auc
            best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
            no_improve = 0
        else:
            no_improve += 1
        
        if no_improve >= patience:
            break
    
    model.load_state_dict(best_state)
    return model, best_auc

def predict_seq_model(model, ids, seqs, device=DEVICE):
    """Generic prediction for GRU or Transformer."""
    ds = EnhancedSeqDataset(ids, seqs, y=None, cad_drop_p=0.0, band_drop_p=0.0)
    dl = DataLoader(ds, batch_size=256, shuffle=False, collate_fn=collate_batch, num_workers=0)
    
    model.eval()
    preds = []
    with torch.no_grad():
        for batch in dl:
            oids, x_num, x_fid, mask = batch
            x_num, x_fid, mask = x_num.to(device), x_fid.to(device), mask.to(device)
            logits, _ = model(x_num, x_fid, mask)
            preds.append(torch.sigmoid(logits).cpu().numpy())
    
    return np.concatenate(preds)

def train_kfold_seq(model_name, model_class, model_params, train_ids, train_seqs, y, 
                    test_ids, test_seqs, n_folds=5, n_seeds=3):
    '''K-fold training with caching support.'''
    
    # Try to load from cache first
    if USE_CACHED_SEQ_MODELS:
        cache = load_seq_cache(model_name, y)
        if cache is not None:
            return cache['oof'], cache['test']
    
    print(f"\n{'='*60}")
    print(f"Training {model_name} ({n_folds} folds, {n_seeds} seeds)")
    print(f"{'='*60}")
    
    n_train = len(train_ids)
    n_test = len(test_ids)
    
    oof_preds = np.zeros(n_train, dtype=np.float64)
    test_preds = np.zeros(n_test, dtype=np.float64)
    
    fold_aucs = []
    
    for seed_idx in range(n_seeds):
        seed = SEED + seed_idx * 77
        set_seed(seed)
        
        skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=seed)
        
        oof_seed = np.zeros(n_train, dtype=np.float64)
        test_seed = np.zeros(n_test, dtype=np.float64)
        
        print(f"\nSeed {seed_idx+1}/{n_seeds} (seed={seed}):")
        
        for fold, (tr_idx, va_idx) in enumerate(skf.split(train_ids, y), 1):
            tr_ids = [train_ids[i] for i in tr_idx]
            va_ids = [train_ids[i] for i in va_idx]
            y_tr = y[tr_idx].astype(np.float32)
            y_va = y[va_idx].astype(np.float32)
            
            model, auc = train_seq_model_fold(
                model_class, model_params, tr_ids, train_seqs, y_tr,
                va_ids, y_va, epochs=100, lr=1e-3, patience=30
            )
            
            oof_seed[va_idx] = predict_seq_model(model, va_ids, train_seqs)
            test_seed += predict_seq_model(model, test_ids, test_seqs) / n_folds
            
            fold_aucs.append(auc)
            print(f"  Fold {fold}: AUC={auc:.4f}")
        
        oof_preds += oof_seed / n_seeds
        test_preds += test_seed / n_seeds
    
    final_auc = roc_auc_score(y, oof_preds)
    print(f"\n{model_name} Results:")
    print(f"  Mean fold AUC: {np.mean(fold_aucs):.4f} ± {np.std(fold_aucs):.4f}")
    print(f"  Final OOF AUC: {final_auc:.4f}")
    
    # Save to cache
    if SAVE_SEQ_MODELS:
        save_seq_cache(model_name, oof_preds, test_preds, fold_aucs)
    
    return oof_preds, test_preds

# Train sequence models
train_ids_list = train_feats["object_id"].tolist()
test_ids_list = test_feats["object_id"].tolist()
y = train_log.set_index("object_id").loc[train_ids_list, "target"].values

# GRU
gru_params = dict(n_filters=6, filt_emb=8, hidden=96, num_in=7, dropout=0.4, n_layers=2)
oof_gru, test_gru = train_kfold_seq("GRU", AttentionGRUEncoder, gru_params,
                                     train_ids_list, train_seqs, y, test_ids_list, test_seqs,
                                     n_folds=N_FOLDS, n_seeds=3)

# Transformer
transformer_params = dict(n_filters=6, filt_emb=8, hidden=96, num_in=7, n_heads=4, n_layers=2, dropout=0.3)
oof_transformer, test_transformer = train_kfold_seq("Transformer", TransformerEncoder, transformer_params,
                                                      train_ids_list, train_seqs, y, test_ids_list, test_seqs,
                                                      n_folds=N_FOLDS, n_seeds=3)

# Compare
best_f1_gru = max(f1_score(y, (oof_gru >= t).astype(int)) for t in np.linspace(0.1, 0.7, 61))
best_f1_transformer = max(f1_score(y, (oof_transformer >= t).astype(int)) for t in np.linspace(0.1, 0.7, 61))
print(f"\nGRU OOF F1: {best_f1_gru:.4f}")
print(f"Transformer OOF F1: {best_f1_transformer:.4f}")

  Loaded GRU cache (OOF AUC: 0.9544, from 2026-01-24T21:44:02.488192)
  Loaded Transformer cache (OOF AUC: 0.8954, from 2026-01-24T23:20:26.479422)

GRU OOF F1: 0.5596
Transformer OOF F1: 0.4167


In [18]:
class PlattScaler:
    def __init__(self):
        self.a = 1.0
        self.b = 0.0
    
    def fit(self, probs, y_true):
        from scipy.special import logit, expit
        from scipy.optimize import minimize
        probs = np.clip(probs, 1e-7, 1 - 1e-7)
        logits = logit(probs)
        def nll(params):
            a, b = params
            scaled = expit(a * logits + b)
            scaled = np.clip(scaled, 1e-7, 1 - 1e-7)
            return -np.mean(y_true * np.log(scaled) + (1 - y_true) * np.log(1 - scaled))
        result = minimize(nll, [1.0, 0.0], method='L-BFGS-B')
        self.a, self.b = result.x
        return self
    
    def calibrate(self, probs):
        from scipy.special import logit, expit
        probs = np.clip(probs, 1e-7, 1 - 1e-7)
        return expit(self.a * logit(probs) + self.b)

In [19]:
# Add GRU and Transformer predictions to features

# Optional: Apply Platt scaling for better calibration
platt_gru = PlattScaler().fit(oof_gru, y)
platt_trans = PlattScaler().fit(oof_transformer, y)

print(f"GRU Platt: a={platt_gru.a:.3f}, b={platt_gru.b:.3f}")
print(f"Transformer Platt: a={platt_trans.a:.3f}, b={platt_trans.b:.3f}")

# Add to train features (OOF predictions)
train_feats['gru_pred'] = platt_gru.calibrate(oof_gru)
train_feats['transformer_pred'] = platt_trans.calibrate(oof_transformer)

# Add to test features
test_feats['gru_pred'] = platt_gru.calibrate(test_gru)
test_feats['transformer_pred'] = platt_trans.calibrate(test_transformer)

print(f"\nTrain features shape: {train_feats.shape}")
print(f"Test features shape: {test_feats.shape}")

# Quick sanity check - verify AUC is preserved
from sklearn.metrics import roc_auc_score
gru_auc = roc_auc_score(y, train_feats['gru_pred'])
trans_auc = roc_auc_score(y, train_feats['transformer_pred'])
print(f"\nGRU AUC (after adding): {gru_auc:.4f}")
print(f"Transformer AUC (after adding): {trans_auc:.4f}")

GRU Platt: a=0.879, b=-2.461
Transformer Platt: a=1.039, b=-2.582

Train features shape: (3043, 156)
Test features shape: (7135, 156)

GRU AUC (after adding): 0.9544
Transformer AUC (after adding): 0.8954


## SECTION 12: TREE MODELS

In [21]:

# Prepare data
feature_cols = [c for c in train_feats.columns if c != 'object_id']
X_train = train_feats[feature_cols].fillna(-999).values
X_test = test_feats[feature_cols].fillna(-999).values
y_train = train_log.set_index('object_id').loc[train_feats['object_id'], 'target'].values

# Optional: StandardScaler (often not needed for tree models)
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

pos = y_train.sum()
neg = len(y_train) - pos
scale_pos_weight = neg / pos

print(f"Features: {len(feature_cols)}")
print(f"Train: {len(X_train)}, Test: {len(X_test)}")
print(f"TDEs: {pos} ({100*pos/len(y_train):.1f}%), scale_pos_weight: {scale_pos_weight:.1f}")

def train_lgb_cv(X, Xt, y, n_folds=5, n_seeds=3):
    params = dict(
        objective="binary", learning_rate=0.03, n_estimators=5000,
        num_leaves=31, max_depth=6, min_child_samples=10,
        subsample=0.8, colsample_bytree=0.8,
        reg_alpha=0.5, reg_lambda=1.0,
        random_state=SEED,
        n_jobs=-1, verbosity=-1
    )
    
    oof = np.zeros(len(y), dtype=np.float64)
    test_pred = np.zeros(len(Xt), dtype=np.float64)
    
    for seed_idx in range(n_seeds):
        seed = SEED + seed_idx * 50
        params["random_state"] = seed
        skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=seed)
        
        for fold, (tr_idx, va_idx) in enumerate(skf.split(X, y), 1):
            model = lgb.LGBMClassifier(**params)
            model.fit(X[tr_idx], y[tr_idx], eval_set=[(X[va_idx], y[va_idx])],
                     callbacks=[lgb.early_stopping(300, verbose=False), lgb.log_evaluation(0)])
            oof[va_idx] += model.predict_proba(X[va_idx])[:, 1] / n_seeds
            test_pred += model.predict_proba(Xt)[:, 1] / (n_folds * n_seeds)
    
    return oof, test_pred

def train_xgb_cv(X, Xt, y, n_folds=5, n_seeds=3):
    params = dict(
        n_estimators=3000,
        learning_rate=0.05,
        max_depth=8,
        min_child_weight=3,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=SEED,
        use_label_encoder=False,
        eval_metric='logloss',
        early_stopping_rounds=100
    )
    
    oof = np.zeros(len(y), dtype=np.float64)
    test_pred = np.zeros(len(Xt), dtype=np.float64)
    
    for seed_idx in range(n_seeds):
        seed = SEED + seed_idx * 50
        params["random_state"] = seed
        skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=seed)
        
        for fold, (tr_idx, va_idx) in enumerate(skf.split(X, y), 1):
            model = xgb.XGBClassifier(**params)
            model.fit(X[tr_idx], y[tr_idx], eval_set=[(X[va_idx], y[va_idx])], verbose=False)
            oof[va_idx] += model.predict_proba(X[va_idx])[:, 1] / n_seeds
            test_pred += model.predict_proba(Xt)[:, 1] / (n_folds * n_seeds)
    
    return oof, test_pred

def train_catboost_cv(X, Xt, y, n_folds=5, n_seeds=3):
    oof = np.zeros(len(y), dtype=np.float64)
    test_pred = np.zeros(len(Xt), dtype=np.float64)
    
    for seed_idx in range(n_seeds):
        seed = SEED + seed_idx * 50
        skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=seed)
        
        for fold, (tr_idx, va_idx) in enumerate(skf.split(X, y), 1):
            model = CatBoostClassifier(
                iterations=5000, learning_rate=0.03, depth=6,
                l2_leaf_reg=3.0, random_strength=0.5, bagging_temperature=0.5,
                early_stopping_rounds=300,
                verbose=False, random_seed=seed
            )
            model.fit(X[tr_idx], y[tr_idx], eval_set=(X[va_idx], y[va_idx]), use_best_model=True)
            oof[va_idx] += model.predict_proba(X[va_idx])[:, 1] / n_seeds
            test_pred += model.predict_proba(Xt)[:, 1] / (n_folds * n_seeds)
    
    return oof, test_pred

print("\nTraining LightGBM...")
oof_lgb, test_lgb = train_lgb_cv(X_train, X_test, y_train)
f1_lgb = max(f1_score(y_train, (oof_lgb >= t).astype(int)) for t in np.linspace(0.1, 0.7, 61))
print(f"  LGB OOF F1: {f1_lgb:.4f}")

print("\nTraining XGBoost...")
oof_xgb, test_xgb = train_xgb_cv(X_train, X_test, y_train)
f1_xgb = max(f1_score(y_train, (oof_xgb >= t).astype(int)) for t in np.linspace(0.1, 0.7, 61))
print(f"  XGB OOF F1: {f1_xgb:.4f}")

print("\nTraining CatBoost...")
oof_cat, test_cat = train_catboost_cv(X_train, X_test, y_train)
f1_cat = max(f1_score(y_train, (oof_cat >= t).astype(int)) for t in np.linspace(0.1, 0.7, 61))
print(f"  CatBoost OOF F1: {f1_cat:.4f}")

# Grid search tree blend weights
print("\nOptimizing tree blend weights...")
best_tree_blend_f1 = 0
best_tree_weights = (1/3, 1/3, 1/3)
best_thr = 0.3

for xgb_w in np.arange(0.0, 1.01, 0.1):
    for lgb_w in np.arange(0.0, 1.01 - xgb_w, 0.1):
        cat_w = 1.0 - xgb_w - lgb_w
        if cat_w < 0:
            continue
        blend = xgb_w * oof_xgb + lgb_w * oof_lgb + cat_w * oof_cat
        for thr in np.linspace(0.1, 0.6, 51):
            f1 = f1_score(y_train, (blend >= thr).astype(int))
            if f1 > best_tree_blend_f1:
                best_tree_blend_f1 = f1
                best_tree_weights = (xgb_w, lgb_w, cat_w)
                best_thr = thr

print(f"Best tree weights: XGB={best_tree_weights[0]:.1f}, LGB={best_tree_weights[1]:.1f}, CAT={best_tree_weights[2]:.1f}")
print(f"Best threshold: {best_thr:.2f}")
print(f"Tree Blend OOF F1: {best_tree_blend_f1:.4f}")

tree_blend_oof = best_tree_weights[0] * oof_xgb + best_tree_weights[1] * oof_lgb + best_tree_weights[2] * oof_cat
tree_blend_test = best_tree_weights[0] * test_xgb + best_tree_weights[1] * test_lgb + best_tree_weights[2] * test_cat

# Final metrics
from sklearn.metrics import precision_score, recall_score
oof_binary = (tree_blend_oof >= best_thr).astype(int)
print(f"\nFinal metrics @ threshold {best_thr:.2f}:")
print(f"  Precision: {precision_score(y_train, oof_binary):.4f}")
print(f"  Recall: {recall_score(y_train, oof_binary):.4f}")
print(f"  Predicted: {oof_binary.sum()} / {y_train.sum()} actual")

Features: 155
Train: 3043, Test: 7135
TDEs: 148 (4.9%), scale_pos_weight: 19.6

Training LightGBM...
  LGB OOF F1: 0.6599

Training XGBoost...
  XGB OOF F1: 0.6726

Training CatBoost...
  CatBoost OOF F1: 0.6372

Optimizing tree blend weights...
Best tree weights: XGB=0.4, LGB=0.4, CAT=0.2
Best threshold: 0.24
Tree Blend OOF F1: 0.6730

Final metrics @ threshold 0.24:
  Precision: 0.6347
  Recall: 0.7162
  Predicted: 167 / 148 actual


## SECTION 13: BLEND WEIGHT OPTIMIZATION

In [23]:
# Define yy (same as y_train from previous cell)
yy = y_train
def evaluate_ensemble(weights, oofs, tests, y):
    """Evaluate weighted ensemble."""
    blend_oof = sum(w * o for w, o in zip(weights, oofs))
    blend_test = sum(w * t for w, t in zip(weights, tests))
    
    best_f1, best_thr = 0, 0.5
    for t in np.linspace(0.1, 0.7, 61):
        f1 = f1_score(y, (blend_oof >= t).astype(int))
        if f1 > best_f1:
            best_f1, best_thr = f1, t
    
    return best_f1, best_thr, blend_oof, blend_test

# First: blend sequence models (GRU + Transformer)
print("\nSequence model blend (GRU vs Transformer):")
best_seq_f1 = 0
best_seq_blend = (0.5, 0.5)
for gru_w in [0.4, 0.5, 0.6, 0.7, 0.8]:
    trans_w = 1 - gru_w
    seq_oof = gru_w * oof_gru + trans_w * oof_transformer
    f1 = max(f1_score(yy, (seq_oof >= t).astype(int)) for t in np.linspace(0.1, 0.7, 61))
    print(f"  GRU={gru_w:.1f}, Trans={trans_w:.1f} -> F1={f1:.4f}")
    if f1 > best_seq_f1:
        best_seq_f1 = f1
        best_seq_blend = (gru_w, trans_w)

print(f"Best seq blend: GRU={best_seq_blend[0]:.1f}, Trans={best_seq_blend[1]:.1f}")
seq_blend_oof = best_seq_blend[0] * oof_gru + best_seq_blend[1] * oof_transformer
seq_blend_test = best_seq_blend[0] * test_gru + best_seq_blend[1] * test_transformer

# Grid search: Tree vs Sequence blend
print("\nGrid search (Tree vs Seq blend):")
print(f"{'Tree_W':<10} {'Seq_W':<10} {'OOF F1':<10} {'Threshold':<10}")
print("-" * 40)

best_config = None
best_f1_overall = 0

for tree_w in np.arange(0.35, 0.75, 0.05):
    seq_w = 1 - tree_w
    f1, thr, _, _ = evaluate_ensemble([tree_w, seq_w], [tree_blend_oof, seq_blend_oof],
                                       [tree_blend_test, seq_blend_test], yy)
    print(f"{tree_w:<10.2f} {seq_w:<10.2f} {f1:<10.4f} {thr:<10.3f}")
    
    if f1 > best_f1_overall:
        best_f1_overall = f1
        best_config = (tree_w, seq_w, thr)

print(f"\nBest: tree_w={best_config[0]:.2f}, seq_w={best_config[1]:.2f}, F1={best_f1_overall:.4f}")

best_tree_w, best_seq_w, best_thr = best_config
final_blend_oof = best_tree_w * tree_blend_oof + best_seq_w * seq_blend_oof
final_blend_test = best_tree_w * tree_blend_test + best_seq_w * seq_blend_test


Sequence model blend (GRU vs Transformer):
  GRU=0.4, Trans=0.6 -> F1=0.5562
  GRU=0.5, Trans=0.5 -> F1=0.5799
  GRU=0.6, Trans=0.4 -> F1=0.5739
  GRU=0.7, Trans=0.3 -> F1=0.5882
  GRU=0.8, Trans=0.2 -> F1=0.5772
Best seq blend: GRU=0.7, Trans=0.3

Grid search (Tree vs Seq blend):
Tree_W     Seq_W      OOF F1     Threshold 
----------------------------------------
0.35       0.65       0.6397     0.610     
0.40       0.60       0.6421     0.580     
0.45       0.55       0.6438     0.520     
0.50       0.50       0.6497     0.500     
0.55       0.45       0.6538     0.480     
0.60       0.40       0.6601     0.480     
0.65       0.35       0.6623     0.460     
0.70       0.30       0.6603     0.410     

Best: tree_w=0.65, seq_w=0.35, F1=0.6623


In [26]:
submissions = {}

# Best blend 
pred_best = (final_blend_test >= best_thr).astype(int)
submissions["best_blend"] = {
    "pred": pred_best,
    "desc": f"tree={best_tree_w:.2f}, seq={best_seq_w:.2f}, thr={best_thr:.3f}",
    "oof_f1": f1_score(yy, (final_blend_oof >= best_thr).astype(int)),
}

# Tree only
tree_best_f1 = max(f1_score(yy, (tree_blend_oof >= t).astype(int)) for t in np.linspace(0.1, 0.7, 61))
tree_best_thr = [t for t in np.linspace(0.1, 0.7, 61) if f1_score(yy, (tree_blend_oof >= t).astype(int)) == tree_best_f1][0]
pred_tree = (tree_blend_test >= tree_best_thr).astype(int)
submissions["tree_only"] = {
    "pred": pred_tree,
    "desc": f"Tree only, thr={tree_best_thr:.3f}",
    "oof_f1": tree_best_f1,
}

# GRU only
gru_best_f1 = max(f1_score(yy, (oof_gru >= t).astype(int)) for t in np.linspace(0.1, 0.7, 61))
gru_best_thr = [t for t in np.linspace(0.1, 0.7, 61) if f1_score(yy, (oof_gru >= t).astype(int)) == gru_best_f1][0]
pred_gru = (test_gru >= gru_best_thr).astype(int)
submissions["gru_only"] = {
    "pred": pred_gru,
    "desc": "GRU only",
    "oof_f1": gru_best_f1,
}

# Print summary
print(f"\\n{'Name':<25} {'OOF F1':<10} {'Positives':<12} {'Rate':<10} {'Description'}")
print("-" * 85)
for name, data in submissions.items():
    rate = data['pred'].sum() / len(data['pred'])
    print(f"{name:<25} {data['oof_f1']:<10.4f} {data['pred'].sum():<12} {rate:<10.4f} {data['desc']}")

# Save all submissions
for name, data in submissions.items():
    sub_df = pd.DataFrame({
        "object_id": test_feats["object_id"].values,
        "target": data["pred"]
    })
    sub_df.to_csv(f"submission_{name}.csv", index=False)

print(f"\\nSaved {len(submissions)} submission files!")

\nName                      OOF F1     Positives    Rate       Description
-------------------------------------------------------------------------------------
best_blend                0.6623     331          0.0464     tree=0.65, seq=0.35, thr=0.460
tree_only                 0.6730     372          0.0521     Tree only, thr=0.240
gru_only                  0.5596     540          0.0757     GRU only
\nSaved 3 submission files!


## SECTION 14: GENERATE SUBMISSIONS

## SECTION 15: SAVE FP/FN FOR ANALYSIS

In [31]:
# Save TP/FN for analysis
# Get tree_only predictions at best threshold
tree_best_f1 = max(f1_score(yy, (tree_blend_oof >= t).astype(int)) for t in np.linspace(0.1, 0.7, 61))
tree_best_thr = [t for t in np.linspace(0.1, 0.7, 61) 
                 if f1_score(yy, (tree_blend_oof >= t).astype(int)) == tree_best_f1][0]

tree_oof_pred = (tree_blend_oof >= tree_best_thr).astype(int)

# Identify TP and FN
tp_mask = (tree_oof_pred == 1) & (yy == 1)  # True Positives
fn_mask = (tree_oof_pred == 0) & (yy == 1)  # False Negatives
fp_mask = (tree_oof_pred == 1) & (yy == 0)  # False Positives
tn_mask = (tree_oof_pred == 0) & (yy == 0)  # True Negatives

print(f"\nTree-only @ threshold={tree_best_thr:.3f}:")
print(f"  True Positives (TP):  {tp_mask.sum()}")
print(f"  False Negatives (FN): {fn_mask.sum()}")
print(f"  False Positives (FP): {fp_mask.sum()}")
print(f"  True Negatives (TN):  {tn_mask.sum()}")
print(f"  F1 Score: {tree_best_f1:.4f}")

# Create DataFrames with object details
tp_objects = train_feats[tp_mask][['object_id']].copy()
tp_objects['type'] = 'TP'
tp_objects['prob'] = tree_blend_oof[tp_mask]

fn_objects = train_feats[fn_mask][['object_id']].copy()
fn_objects['type'] = 'FN'
fn_objects['prob'] = tree_blend_oof[fn_mask]

fp_objects = train_feats[fp_mask][['object_id']].copy()
fp_objects['type'] = 'FP'
fp_objects['prob'] = tree_blend_oof[fp_mask]

# Combine and add useful features for analysis
tree_only_analysis = pd.concat([tp_objects, fn_objects, fp_objects], ignore_index=True)

# Add key features to help understand misclassifications
key_features = ['Z', 'n_detected', 'flux_max', 'decay_alpha', 'gru_pred', 
                'trans_delta_m15', 'sn_is_good_ia_fit']
key_features = [f for f in key_features if f in train_feats.columns]

for feat in key_features:
    tree_only_analysis = tree_only_analysis.merge(
        train_feats[['object_id', feat]], on='object_id', how='left'
    )

# Save to CSV
tree_only_analysis.to_csv('tree_only_tp_fn_fp.csv', index=False)
print(f"\nSaved: tree_only_tp_fn_fp.csv")

# Also save just TP and FN for focused analysis
tp_fn_only = tree_only_analysis[tree_only_analysis['type'].isin(['TP', 'FN'])]
tp_fn_only.to_csv('tree_only_tp_fn.csv', index=False)
print(f"Saved: tree_only_tp_fn.csv ({len(tp_fn_only)} objects)")

# Show FN analysis (these are TDEs we're missing)
print("\n" + "-"*50)
print("FALSE NEGATIVE ANALYSIS (TDEs we're missing):")
print("-"*50)
fn_df = tree_only_analysis[tree_only_analysis['type'] == 'FN']
print(f"Total FN: {len(fn_df)}")
print(f"\nFN probability distribution:")
print(f"  Mean: {fn_df['prob'].mean():.4f}")
print(f"  Std:  {fn_df['prob'].std():.4f}")
print(f"  Min:  {fn_df['prob'].min():.4f}")
print(f"  Max:  {fn_df['prob'].max():.4f}")

# Show borderline FNs (close to threshold)
borderline_fn = fn_df[fn_df['prob'] >= tree_best_thr - 0.1]
print(f"\nBorderline FN (prob >= {tree_best_thr-0.1:.2f}): {len(borderline_fn)}")
if len(borderline_fn) > 0:
    print(borderline_fn[['object_id', 'prob'] + key_features[:3]].head(10).to_string())




Tree-only @ threshold=0.240:
  True Positives (TP):  106
  False Negatives (FN): 42
  False Positives (FP): 61
  True Negatives (TN):  2834
  F1 Score: 0.6730

Saved: tree_only_tp_fn_fp.csv
Saved: tree_only_tp_fn.csv (148 objects)

--------------------------------------------------
FALSE NEGATIVE ANALYSIS (TDEs we're missing):
--------------------------------------------------
Total FN: 42

FN probability distribution:
  Mean: 0.0899
  Std:  0.0653
  Min:  0.0050
  Max:  0.2196

Borderline FN (prob >= 0.14): 11
                 object_id      prob        Z  n_detected  decay_alpha
112      amon_corch_idhren  0.162559  0.38890          51     0.238933
114   anann_ungol_agarwaen  0.219566  0.08658          48     0.608426
120   eilian_perian_glamor  0.206508  0.46410          53     0.458275
124  gaurwaith_tamin_ungol  0.179621  0.31120          53     0.585562
125   gwarth_bellas_glamor  0.168036  0.32490          40     1.056391
129    hwiniol_randir_nell  0.181065  0.48610          5